

---
#**TCO: Tarea 2- EL VIAJANTE DE COMERCIO**
 Alexander Olza Rodriguez


---




**Definición del problema:**

El problema del viajante de comercio (TSP) halla el recorrido óptimo entre los nodos de un grafo exigiendo las siguientes propiedades:


*   Todos los nodos han de visitarse una y sólo una vez
*   El recorrido ha de ser cerrado, volviendo al final al nodo de origen

Esto implica que no puede haber subtours.

Habitualmente se busca la solución que minimice una matriz de costes dada. En una aplicación práctica, los elementos $c_{ij}$ de esta matriz pueden ser distancias entre ciudades en una ruta, pero también tiene otras aplicaciones (por ejemplo, EJEMPLO).





In [ ]:
!pip install ortools
from ortools.linear_solver import pywraplp

In [ ]:
n     = 25  #Con 50 no se ejecuta bien, a pesar de que el modelo está bien planteado
todos = range(n)
otros = range(1,n)
EPS   = 0.00001

import random
random.seed(12345)
distancias  = { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }
print('done')

done


Para introducir el modelo que he elegido, podemos considerar el TSP de la siguiente manera:

Imaginemos un grupo de $n-1$ viajeros de $n-1$ nacionalidades distintas (por lo tanto son viajeros distinguibles) a bordo de un autobús. El autobús ha de recorrer todos los países, dejando a cada viajero en su hogar, y volver vacío al punto de partida, de forma que en total visite $n$ lugares.

Esto es sólo una interpretación, y las siguientes variables son aplicables al TSP en general.

Sean estas las variables del problema:

$ x_{ij} = \left \{
\begin{array}{ll}
      1 & \textrm{si viajo por la carretera } (i,j) \\
      0 & \textrm{en otro caso} \\
\end{array} 
\right.$ \\

$ f_{ij}^k = \left \{
\begin{array}{ll}
      1 & \textrm{si el viajero $k$ sigue a bordo cuando voy por la carretera } (i,j) \\
      0 & \textrm{en otro caso} \\
\end{array} 
\right.$ \\


In [ ]:
#solver =pywraplp.Solver('ATSP2xIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#solver=pywraplp.Solver.CreateSolver('CBC')
#solver= pywraplp.Solver('ViajerosInternacionales',  pywraplp.Solver.BOP_INTEGER_PROGRAMMING )#el limite es 14 con opt 132, 0.591  seconds
solver = pywraplp.Solver.CreateSolver('SCIP') #25 nodes 25.79  seconds, optimal 165: vs profesor con CBC modelo con  f opt= 165.0  en  0.566 seconds
x = { (i,j) : solver.BoolVar('carretera[%i, %i]' % (i, j)) for i in todos for j in todos if j!=i}
f = {(i,j,k): solver.BoolVar('presente[%i,%i,%i]'%(k,i,j)) for k in otros for i in todos for j in todos if j!=i}
print('ok')

ok


La función objetivo a minimizar podría ser la distancia por carretera en toda la ruta. No tiene por qué ser simétrica, ya que podría tratarse de carreteras unidireccionales. Naturalmente, sólo suman las distancias de las carreteras por las que elija transitar, y esto se manifiesta en el producto por $x_{ij}$.

**minimizar** $\sum_{i,j\ne i}c_{ij}x_{ij}$

In [ ]:
solver.Minimize(solver.Sum( distancias[i,j]*x[i,j] for i,j in distancias.keys()))
print('ok')

ok


Ahora planteamos las restricciones del modelo.



* **A)** Desde cada origen $i$ hay que salir hacia algún destino $j$:

  $\sum_j x_{ij}=1 \quad\forall i$
* **B)** A cada destino $j$ hay que llegar desde algún origen $i$:

  $\sum_i x_{ij}=1 \quad\forall j$


* **C)**  Al principio todos los viajeros ($\forall k$) salen del origen ($i=0$) hacia algún lugar desconocido $j$:

    $\sum_{j} f_{0j}^k=1\quad \forall k$

* **D)** Ningún viajero vuelve al lugar de origen ($j=0$) al final del tour: 

  $\sum_{i} f_{i0}^k=0\quad \forall k$



* **E)** Cuando un pasajero llega a su país ($i=k$), se queda en él (no sale hacia ningún destino $j$):

  $\sum_{j\ne k} f_{kj}^k=0\quad \forall k$
* **F)** Si el pasajero $k$ entra a un país $i\ne k$ que no es el suyo (y puede haber llegado desde cualquier $j$), también saldrá de $i$ hacia algún otro país $j$:

  $\sum_jf_{ji}^k=\sum_jf_{ij}^k\quad \forall i\ne k, i\ne 1; \forall k$

* **G)** Conexión entre $f_{ji}^k$ y $x_{ij}$: 

  Si el autobús no va por la carretera $(i,j)$, ningún viajero va por esa carretera: $x_{ij}=0\Rightarrow f_{ji}^k=0 \forall k$. Si el autobús va por esa carretera, el viajero $k$ puede estar presente o no: $x_{ij}=1\Rightarrow f_{ji}^k=0, 1$. En conjunto,

  $f_{ji}^k\le x_{ij}$



A continuación implementamos estas restricciones.



In [ ]:
[ solver.Add( solver.Sum( x[i,j] for j in todos if j!=i) == 1 , name='A')  for i in todos ]
[ solver.Add( solver.Sum( x[i,j] for i in todos if j!=i) == 1 , name='B')  for j in todos ]
[ solver.Add( solver.Sum(f[0,j,k] for j in otros) ==1 , name='C')  for k in otros ]
[ solver.Add( solver.Sum(f[i,0,k] for i in otros if i!=k) ==0 , name='D')  for k in otros ]
[ solver.Add( solver.Sum(f[k,j,k] for j in todos if j!=k) ==0 , name='E')  for k in otros ]
[ solver.Add( solver.Sum(f[j,i,k] for j in todos if j!=i) == solver.Sum(f[i,j,k] for j in todos if j!=i) , name='F')  for i in otros for k in otros if i!=k ]#19
[ solver.Add( f[i,j,k]<=x[i,j] ,name='G') for k in otros for i in todos for j in todos if j!=i ]
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('ok')

Number of variables = 15000
Number of constraints = 15074
ok


Por último, se resuelve el problema y se muestra el tour elegido:

In [ ]:

solver.Solve()
#assert result_status == pywraplp.Solver.OPTIMAL
  
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print( solver.WallTime()/1000, " seconds")
print('Optimal objective value = %f' % solver.Objective().Value())
  
selected = []
for i in todos:
    for j in todos:
       if j!=i:
            if x[i,j].solution_value() > EPS :
              selected.append((i,j))
print('Optimal path: %s' % str(selected))
print('ok')

Number of variables = 15000
Number of constraints = 15074
29.866  seconds
Optimal objective value = 165.000000
Optimal path: [(0, 24), (1, 17), (2, 6), (3, 23), (4, 7), (5, 11), (6, 0), (7, 13), (8, 4), (9, 22), (10, 1), (11, 15), (12, 19), (13, 10), (14, 16), (15, 20), (16, 8), (17, 5), (18, 12), (19, 14), (20, 3), (21, 9), (22, 2), (23, 21), (24, 18)]
ok
